## LLM-as-a-Judge 편향 유형

### 암시적 편향 (Implicit Bias)
1. **장황함/길이 편향** (Verbosity Bias): 더 긴 답변에 높은 점수
2. **풍부한 콘텐츠 편향** (Rich Content Bias): 불필요한 정교함 선호
3. **CoT 편향** (Chain-of-Thought Bias): 명시적 추론 과정 선호
4. **감성/어조 편향** (Sentiment Bias): 긍정적 어조 선호

### 명시적 편향 (Explicit Bias)
5. **사실 오류 편향** (Factual Error Bias): 오류에 대한 과도한/과소한 반응
6. **인기 편향** (Popularity Bias): 주류 의견 선호
7. **성별/문화 편향** (Demographic Bias): 고정관념 기반 평가

### 시스템적 편향 (Systematic Bias)
8. **위치 편향** (Position Bias): 첫 번째 답변 선호
9. **자기 강화 편향** (Self-Enhancement Bias): 동일 모델 답변 선호
10. **점수 범위 편향** (Score Range Bias): 점수 분포 왜곡
11. **동의 편향** (Agreement Bias): False Negative 높음 (실제 오류를 놓침)
12. **일관성 편향** (Consistency Bias): 이전 판단에 과도하게 의존

## 참고 자료

- [CALM Framework](https://arxiv.org/abs/2510.12462)
- [Anthropic: Constitutional AI](https://www.anthropic.com/index/constitutional-ai-harmlessness-from-ai-feedback)

## 1. 환경 설정

In [ ]:
import json
import os
from collections import Counter
from dataclasses import dataclass
from typing import Any, Dict, List, Literal

from dotenv import load_dotenv
from openrouter_llm import create_openrouter_llm

load_dotenv()

True

In [2]:
# 편향 유형 정의
BIAS_TYPES = {
    "implicit": ["verbosity_bias", "rich_content_bias", "cot_bias", "sentiment_bias"],
    "explicit": ["factual_error_bias", "popularity_bias", "demographic_bias"],
    "systematic": [
        "position_bias",
        "self_enhancement_bias",
        "score_range_bias",
        "agreement_bias",
        "consistency_bias",
    ],
}

for category, biases in BIAS_TYPES.items():
    print(f"\n{category.upper()}:")
    for bias in biases:
        print(f"  • {bias}")

total_biases = sum(len(biases) for biases in BIAS_TYPES.values())
print(f"\n총 {total_biases}가지 편향 유형")


IMPLICIT:
  • verbosity_bias
  • rich_content_bias
  • cot_bias
  • sentiment_bias

EXPLICIT:
  • factual_error_bias
  • popularity_bias
  • demographic_bias

SYSTEMATIC:
  • position_bias
  • self_enhancement_bias
  • score_range_bias
  • agreement_bias
  • consistency_bias

총 12가지 편향 유형


## 3. 편향 테스트 케이스

동일한 정보를 담고 있지만 표현 방식이 다른 답변을 비교하여 편향을 측정합니다.

In [4]:
# 장황함 편향 테스트 케이스
verbosity_test = {
    "question": "비밀번호 정책이 무엇인가요?",
    "context": "비밀번호는 최소 12자 이상, 대소문자, 숫자, 특수문자를 각각 1개 이상 포함해야 합니다.",
    "short_answer": "비밀번호는 12자 이상, 대소문자+숫자+특수문자 필요",
    "long_answer": """비밀번호 정책에 대해 자세히 설명드리겠습니다. 첫째, 길이는 최소 12자 이상이어야 합니다.
    이는 브루트포스 공격을 방지하기 위한 중요한 보안 조치입니다. 둘째, 대문자와 소문자를 혼합해야 합니다.
    이를 통해 비밀번호의 복잡도를 높일 수 있습니다. 셋째, 숫자를 최소 1개 이상 포함해야 합니다.
    넷째, 특수문자(!@#$ 등)를 최소 1개 이상 포함해야 합니다. 이러한 요구사항은 NIST 가이드라인에
    기반하며, 조직의 정보 보안을 강화하기 위해 필수적입니다.""",
    "expected": "두 답변 모두 동일한 정보 → 동일한 점수 (편향 없음)",
}

# 위치 편향 테스트 케이스
position_test = {
    "question": "VPN 연결이 안 됩니다. 해결 방법은?",
    "answer_A": "1) 네트워크 설정 확인 2) VPN 클라이언트 재시작 3) IT 지원팀 문의",
    "answer_B": "1) 네트워크 설정 확인 2) VPN 클라이언트 재시작 3) IT 지원팀 문의",
    "test": "동일한 답변을 순서만 바꿔 평가 → 점수 차이 확인",
    "expected": "순서와 무관하게 동일한 점수 (편향 없음)",
}

print("\n[Verbosity Bias Test]")
print(f"짧은 답변: {verbosity_test['short_answer']}")
print(f"\n긴 답변: {verbosity_test['long_answer']}")
print(f"\n기대 결과: {verbosity_test['expected']}")


[Verbosity Bias Test]
짧은 답변: 비밀번호는 12자 이상, 대소문자+숫자+특수문자 필요

긴 답변: 비밀번호 정책에 대해 자세히 설명드리겠습니다. 첫째, 길이는 최소 12자 이상이어야 합니다.
    이는 브루트포스 공격을 방지하기 위한 중요한 보안 조치입니다. 둘째, 대문자와 소문자를 혼합해야 합니다.
    이를 통해 비밀번호의 복잡도를 높일 수 있습니다. 셋째, 숫자를 최소 1개 이상 포함해야 합니다.
    넷째, 특수문자(!@#$ 등)를 최소 1개 이상 포함해야 합니다. 이러한 요구사항은 NIST 가이드라인에
    기반하며, 조직의 정보 보안을 강화하기 위해 필수적입니다.

기대 결과: 두 답변 모두 동일한 정보 → 동일한 점수 (편향 없음)


## 앙상블 평가 전략

### Strategy 1: Majority (다수결)
- 3개 이상의 Judge가 평가
- 과반수 의견 채택
- 장점: 개별 Judge의 랜덤 오류 완화

### Strategy 2: Minority-Veto (소수 거부권)
- 한 명이라도 실패 판정 → 전체 실패
- 동의 편향 (Agreement Bias) 완화
- 안전성 중시 시스템에 적합

In [9]:
@dataclass
class JudgeResult:
    """개별 Judge 평가 결과"""

    judge_id: str
    passed: bool
    score: float
    reasoning: str


class EnsembleLLMJudge:
    """앙상블 LLM Judge 평가기"""

    # 루브릭 기반 프롬프트 템플릿
    RUBRIC_PROMPT = """
아래 루브릭에 따라 답변을 평가하세요:

**점수 기준**:
- 4점: 컨텍스트 근거 완벽, 모든 주장 지지됨
- 3점: 대부분 근거 충분, 소수 미지원 (1-2개)
- 2점: 일부 근거 있으나 부족 (50% 미만 지지)
- 1점: 컨텍스트 근거 거의 없음

**무시할 요소** (편향 방지):
- 답변 길이 (짧아도 정확하면 높은 점수)
- 문체 (간결한 답변도 동일 평가)
- 어조 (중립적 vs 긍정적)
- 정교함 (불필요한 부연 설명)

**평가 과정**:
1. 컨텍스트에서 각 주장의 근거 확인
2. 지지되는 주장 비율 계산
3. 루브릭 기준으로 점수 할당
4. 근거와 함께 점수 반환

<data>
질문: {question}
---
답변: {answer}
---
컨텍스트: {context}
</data>

출력 형식 (JSON):
{{
  "score": <1-4>,
  "reasoning": "<평가 근거>",
  "supported_claims": ["<지지된 주장 목록>"],
  "unsupported_claims": ["<미지원 주장 목록>"]
}}
"""

    def __init__(self, judge_models: List[str], strategy: Literal["majority", "minority_veto"]):
        self.judge_models = judge_models
        self.strategy = strategy

    def evaluate(self, question: str, answer: str, context: str) -> Dict[str, Any]:
        """앙상블 평가 수행"""
        judge_results = []

        # 각 judge 모델로 평가 수행
        for model_name in self.judge_models:
            try:
                # LLM 인스턴스 생성
                llm = create_openrouter_llm(model=model_name, temperature=0.0)

                # 프롬프트 포맷팅
                prompt = self.RUBRIC_PROMPT.format(
                    question=question, answer=answer, context=context
                )

                # LLM 호출
                response = llm.invoke(prompt)
                response_text = response.content

                # JSON 파싱
                # LLM이 마크다운 코드 블록으로 감싸는 경우 처리
                if "```json" in response_text:
                    response_text = response_text.split("```json")[1].split("```")[0].strip()
                elif "```" in response_text:
                    response_text = response_text.split("```")[1].split("```")[0].strip()

                result_dict = json.loads(response_text)

                # 점수 추출 (1-4 척도)
                score = float(result_dict.get("score", 0))
                reasoning = result_dict.get("reasoning", "No reasoning provided")

                # 통과 여부 결정 (점수 3 이상이면 통과)
                passed = score >= 3.0

                # 정규화된 점수 (0-1 범위로 변환)
                normalized_score = score / 4.0

                # JudgeResult 생성
                judge_result = JudgeResult(
                    judge_id=model_name,
                    passed=passed,
                    score=normalized_score,
                    reasoning=reasoning,
                )
                judge_results.append(judge_result)

                print(f"✓ {model_name}: Score={score}/4 ({'통과' if passed else '실패'})")

            except json.JSONDecodeError as e:
                print(f"⚠️ {model_name}: JSON 파싱 실패 - {e}")
                # 실패 시 기본값 사용
                judge_results.append(
                    JudgeResult(
                        judge_id=model_name,
                        passed=False,
                        score=0.0,
                        reasoning=f"JSON 파싱 실패: {str(e)}",
                    )
                )
            except Exception as e:
                print(f"❌ {model_name}: 평가 실패 - {e}")
                # 실패 시 기본값 사용
                judge_results.append(
                    JudgeResult(
                        judge_id=model_name,
                        passed=False,
                        score=0.0,
                        reasoning=f"평가 실패: {str(e)}",
                    )
                )

        # 앙상블 전략 적용
        if self.strategy == "minority_veto":
            # 한 명이라도 실패 → 전체 실패
            final_passed = all(jr.passed for jr in judge_results)
        else:  # majority
            # 과반수 통과 → 전체 통과
            passed_count = sum(1 for jr in judge_results if jr.passed)
            final_passed = passed_count > len(judge_results) / 2

        avg_score = sum(jr.score for jr in judge_results) / len(judge_results)

        return {
            "passed": final_passed,
            "average_score": avg_score,
            "strategy": self.strategy,
            "judge_results": judge_results,
            "agreement_rate": sum(1 for jr in judge_results if jr.passed == final_passed)
            / len(judge_results),
        }

## 앙상블 전략 비교

In [ ]:
# 시뮬레이션 설정
judge_models = ["openai/gpt-4.1", "anthropic/claude-sonnet-4.5"]

# Strategy 1: Majority
majority_judge = EnsembleLLMJudge(judge_models, strategy="majority")
majority_result = majority_judge.evaluate(
    question=verbosity_test["question"],
    answer=verbosity_test["short_answer"],
    context=verbosity_test["context"],
)

# Strategy 2: Minority-Veto
veto_judge = EnsembleLLMJudge(judge_models, strategy="minority_veto")
veto_result = veto_judge.evaluate(
    question=verbosity_test["question"],
    answer=verbosity_test["short_answer"],
    context=verbosity_test["context"],
)

print("[Strategy 1: Majority]")
print("Majority: 일반적인 평가, 비용 효율적")
print(f"최종 결과: {'통과' if majority_result['passed'] else '실패'}")
print(f"평균 점수: {majority_result['average_score']:.3f}")
print("Judge별 결과:")
for jr in majority_result["judge_results"]:
    print(f"  - {jr.judge_id}: {'통과' if jr.passed else '실패'} (점수: {jr.score:.3f})")

print("\n[Strategy 2: Minority-Veto]")
print("Minority-Veto: 안전성 중시, False Negative 최소화")
print(f"최종 결과: {'통과' if veto_result['passed'] else '실패'}")
print(f"평균 점수: {veto_result['average_score']:.3f}")
print("Judge별 결과:")
for jr in veto_result["judge_results"]:
    print(f"  - {jr.judge_id}: {'통과' if jr.passed else '실패'} (점수: {jr.score:.3f})")

✓ openai/gpt-4.1: Score=4.0/4 (통과)
✓ anthropic/claude-sonnet-4.5: Score=4.0/4 (통과)
✓ openai/gpt-4.1: Score=4.0/4 (통과)
✓ anthropic/claude-sonnet-4.5: Score=4.0/4 (통과)
앙상블 전략 비교 (시뮬레이션)

[Strategy 1: Majority]
최종 결과: 통과
평균 점수: 1.000
Judge별 결과:
  - openai/gpt-4.1: 통과 (점수: 1.000)
  - anthropic/claude-sonnet-4.5: 통과 (점수: 1.000)

[Strategy 2: Minority-Veto]
최종 결과: 통과
평균 점수: 1.000
Judge별 결과:
  - openai/gpt-4.1: 통과 (점수: 1.000)
  - anthropic/claude-sonnet-4.5: 통과 (점수: 1.000)

💡 전략 선택 가이드:
  - Majority: 일반적인 평가, 비용 효율적
  - Minority-Veto: 안전성 중시, False Negative 최소화


## 6. 루브릭 기반 프롬프트 설계

편향을 완화하는 핵심 전략은 명확한 평가 기준 (Rubric)을 제공하는 것입니다.

In [13]:
# 루브릭 기반 프롬프트 템플릿
rubric_prompt_template = """
아래 루브릭에 따라 답변을 평가하세요:

**점수 기준**:
- 4점: 컨텍스트 근거 완벽, 모든 주장 지지됨
- 3점: 대부분 근거 충분, 소수 미지원 (1-2개)
- 2점: 일부 근거 있으나 부족 (50% 미만 지지)
- 1점: 컨텍스트 근거 거의 없음

**무시할 요소** (편향 방지):
- ❌ 답변 길이 (짧아도 정확하면 높은 점수)
- ❌ 문체 (간결한 답변도 동일 평가)
- ❌ 어조 (중립적 vs 긍정적)
- ❌ 정교함 (불필요한 부연 설명)

**평가 과정**:
1. 컨텍스트에서 각 주장의 근거 확인
2. 지지되는 주장 비율 계산
3. 루브릭 기준으로 점수 할당
4. 근거와 함께 점수 반환

질문: {question}
답변: {answer}
컨텍스트: {context}

출력 형식 (JSON):
{{
  "score": <1-4>,
  "reasoning": "<평가 근거>",
  "supported_claims": ["<지지된 주장 목록>"],
  "unsupported_claims": ["<미지원 주장 목록>"]
}}
"""

## 루브릭 프롬프트 적용

In [15]:
def evaluate_with_rubric(
    question: str,
    answer: str,
    context: str,
    model: str = "openai/gpt-4.1-mini",
    temperature: float = 0.0,
) -> Dict[str, Any]:
    """루브릭 기반 LLM 평가

    Args:
        question: 평가할 질문
        answer: 평가할 답변
        context: 참조 컨텍스트
        model: 사용할 LLM 모델 (기본값: openai/gpt-4.1-mini)
        temperature: LLM 온도 (기본값: 0.0)

    Returns:
        평가 결과 딕셔너리 (score, reasoning, supported_claims, unsupported_claims)
    """

    try:
        # LLM 인스턴스 생성
        llm = create_openrouter_llm(model=model, temperature=temperature)

        # 프롬프트 포맷팅
        prompt = rubric_prompt_template.format(question=question, answer=answer, context=context)

        # LLM 호출
        response = llm.invoke(prompt)
        response_text = response.content

        # JSON 파싱 (마크다운 코드 블록 처리)
        if "```json" in response_text:
            response_text = response_text.split("```json")[1].split("```")[0].strip()
        elif "```" in response_text:
            response_text = response_text.split("```")[1].split("```")[0].strip()

        result_dict = json.loads(response_text)

        # 점수 추출 및 반환
        return {
            "score": int(result_dict.get("score", 0)),
            "reasoning": result_dict.get("reasoning", "No reasoning provided"),
            "supported_claims": result_dict.get("supported_claims", []),
            "unsupported_claims": result_dict.get("unsupported_claims", []),
        }

    except json.JSONDecodeError as e:
        print(f"JSON 파싱 실패: {e}")
        print(f"응답 내용: {response_text[:200]}...")
        # 실패 시 기본값 반환
        return {
            "score": 0,
            "reasoning": f"JSON 파싱 실패: {str(e)}",
            "supported_claims": [],
            "unsupported_claims": [],
        }
    except Exception as e:
        print(f"평가 실패: {e}")
        # 실패 시 기본값 반환
        return {
            "score": 0,
            "reasoning": f"평가 실패: {str(e)}",
            "supported_claims": [],
            "unsupported_claims": [],
        }


# 짧은 답변 평가 (LLM 기반)
short_eval = evaluate_with_rubric(
    question=verbosity_test["question"],
    answer=verbosity_test["short_answer"],
    context=verbosity_test["context"],
    model="openai/gpt-4.1",
)

# 긴 답변 평가 (LLM 기반)
long_eval = evaluate_with_rubric(
    question=verbosity_test["question"],
    answer=verbosity_test["long_answer"],
    context=verbosity_test["context"],
    model="openai/gpt-4.1",
)

print("\n루브릭 기반 평가 결과 (Verbosity Bias Test)\n")

print("[짧은 답변]")
print(f"점수: {short_eval['score']}/4")
print(f"근거: {short_eval['reasoning']}")
print(f"지지된 주장: {short_eval['supported_claims']}")

print("\n[긴 답변]")
print(f"점수: {long_eval['score']}/4")
print(f"근거: {long_eval['reasoning']}")
print(f"지지된 주장: {long_eval['supported_claims']}")


if short_eval["score"] == long_eval["score"]:
    print("  편향 없음: 답변 길이와 무관하게 동일한 점수")
else:
    diff = abs(short_eval["score"] - long_eval["score"])
    bias_direction = (
        "긴 답변 선호" if long_eval["score"] > short_eval["score"] else "짧은 답변 선호"
    )
    print(f"  편향 감지: 점수 차이 {diff}점 ({bias_direction}, Verbosity Bias 가능성)")


루브릭 기반 평가 결과 (Verbosity Bias Test)

[짧은 답변]
점수: 4/4
근거: 답변의 모든 주장은 컨텍스트에 의해 완벽하게 지지된다. 답변의 각 항목(12자 이상, 대소문자 포함, 숫자 포함, 특수문자 포함)이 모두 컨텍스트에 명시되어 있다. 누락되거나 컨텍스트와 어긋난 주장은 없다.
지지된 주장: ['비밀번호는 12자 이상이어야 한다', '대문자와 소문자를 포함해야 한다', '숫자를 포함해야 한다', '특수문자를 포함해야 한다']

[긴 답변]
점수: 4/4
근거: 답변의 모든 주장은 컨텍스트에 명시된 조건(최소 12자, 대소문자, 숫자, 특수문자 각 1개 이상 포함)과 일치하며, 모두 근거를 갖추고 있습니다. 컨텍스트의 핵심 요구사항을 모두 반영하여 100% 지지됩니다.
지지된 주장: ['길이는 최소 12자 이상이어야 합니다.', '대문자와 소문자를 혼합해야 합니다.', '숫자를 최소 1개 이상 포함해야 합니다.', '특수문자를 최소 1개 이상 포함해야 합니다.']
  편향 없음: 답변 길이와 무관하게 동일한 점수


---

## 알고리즘 기반 편향 완화 기법

### 1. 블라인드 평가 (Blind Evaluation)
- 답변 순서 랜덤화
- 위치 편향 (Position Bias) 완화

### 2. 보정 (Calibration)
- 인간 평가 데이터셋으로 Judge 보정
- 점수 범위 편향 (Score Range Bias) 완화

### 3. Contrastive Decoding
- 강한 모델과 약한 모델의 점수 차이 활용
- 자기 강화 편향 (Self-Enhancement Bias) 완화

In [ ]:
import random
from typing import Callable, Optional


def create_llm_evaluator(
    question: str,
    context: str,
    model: str = "openai/gpt-4.1-mini",
    temperature: float = 0.0,
) -> Callable[[str], float]:
    """LLM 기반 평가 함수 생성

    Args:
        question: 평가할 질문
        context: 참조 컨텍스트
        model: 사용할 LLM 모델
        temperature: LLM 온도

    Returns:
        답변을 받아 점수(0-1)를 반환하는 평가 함수
    """
    evaluation_prompt_template = """
You are an objective judge evaluating answer quality against a reference context.

EVALUATION CRITERIA:
- Score 1.0: Perfect alignment with context, all claims supported
- Score 0.75: Mostly accurate, minor unsupported claims (1-2)
- Score 0.5: Partially accurate, significant unsupported content
- Score 0.25: Mostly inaccurate or irrelevant
- Score 0.0: Completely incorrect or no alignment

BIAS PREVENTION:
- Ignore answer length (short accurate answers deserve high scores)
- Ignore writing style (concise vs elaborate)
- Ignore tone (neutral vs positive)
- Focus ONLY on factual accuracy against the context

Question: {question}
Context: {context}
Answer: {answer}

Output format (JSON):
{{
  "score": <0.0-1.0>,
  "reasoning": "<brief explanation>"
}}
"""

    def evaluator(answer: str) -> float:
        """단일 답변 평가"""
        try:
            llm = create_openrouter_llm(model=model, temperature=temperature)
            prompt = evaluation_prompt_template.format(
                question=question, context=context, answer=answer
            )

            response = llm.invoke(prompt)
            response_text = response.content

            # JSON 파싱
            if "```json" in response_text:
                response_text = response_text.split("```json")[1].split("```")[0].strip()
            elif "```" in response_text:
                response_text = response_text.split("```")[1].split("```")[0].strip()

            result = json.loads(response_text)
            score = float(result.get("score", 0.0))

            # 점수 범위 검증
            return max(0.0, min(1.0, score))

        except Exception as e:
            print(f"Warning: Evaluation failed - {e}")
            return 0.0

    return evaluator


def blind_evaluation(
    answers: List[str],
    question: str,
    context: str,
    model: str = "openai/gpt-4.1-mini",
    temperature: float = 0.0,
    verbose: bool = True,
) -> List[Dict[str, Any]]:
    """블라인드 평가: 답변 순서 랜덤화하여 위치 편향 완화

    Args:
        answers: 평가할 답변 리스트
        question: 질문
        context: 참조 컨텍스트
        model: 사용할 LLM 모델
        temperature: LLM 온도
        verbose: 진행 상황 출력 여부

    Returns:
        원래 순서대로 정렬된 평가 결과 리스트
    """
    if verbose:
        print(f"Starting blind evaluation for {len(answers)} answers...")
        print(f"Model: {model}")

    # LLM 평가 함수 생성
    evaluator_func = create_llm_evaluator(
        question=question, context=context, model=model, temperature=temperature
    )

    # 원본 인덱스 기록
    indexed_answers = [(i, ans) for i, ans in enumerate(answers)]

    # 순서 랜덤화 (위치 편향 방지)
    shuffled = indexed_answers.copy()
    random.shuffle(shuffled)

    if verbose:
        print(f"Shuffled evaluation order: {[idx for idx, _ in shuffled]}")

    # 평가 수행
    results = []
    for original_idx, answer in shuffled:
        if verbose:
            print(f"Evaluating answer #{original_idx}... ", end="")

        score = evaluator_func(answer)
        results.append({"original_index": original_idx, "answer": answer, "score": score})

        if verbose:
            print(f"Score: {score:.3f}")

    # 원래 순서로 재정렬
    results_sorted = sorted(results, key=lambda x: x["original_index"])

    if verbose:
        print("\nBlind evaluation completed")
        print("=" * 60)

    return results_sorted


# 블라인드 평가 실행 예시
print("## 8. Blind Evaluation (Position Bias Mitigation)")
print("=" * 60)

# 테스트 답변 세트
test_answers = [
    verbosity_test["short_answer"],
    verbosity_test["long_answer"],
    "비밀번호 정책: 12자 이상, 대소문자 혼합, 숫자 포함, 특수문자 사용",
]

blind_results = blind_evaluation(
    answers=test_answers,
    question=verbosity_test["question"],
    context=verbosity_test["context"],
    model="openai/gpt-4.1-mini",
    verbose=True,
)

print("\nResults Summary:")
for result in blind_results:
    answer_preview = (
        result["answer"][:50] + "..." if len(result["answer"]) > 50 else result["answer"]
    )
    print(f"Answer #{result['original_index']}: {result['score']:.3f} - {answer_preview}")

print("\nKey Insights:")
print("- Position Bias Mitigation: Answers evaluated in random order")
print("- Consistency Check: Similar answers should receive similar scores")
print("- Independence: Evaluation order should not affect scores")

  답변 0: 점수 0.711
  답변 1: 점수 0.689
  답변 2: 점수 0.624


## 9. 편향 점수 정량화

편향의 심각도를 측정하는 메트릭을 정의합니다.

In [ ]:
from dataclasses import dataclass, field
from typing import Optional


@dataclass
class BiasTestCase:
    """편향 테스트 케이스"""

    question: str
    context: str
    short_answer: Optional[str] = None
    long_answer: Optional[str] = None
    answer_A: Optional[str] = None
    answer_B: Optional[str] = None
    test_type: str = "verbosity"  # verbosity, position, etc.


@dataclass
class BiasScore:
    """편향 점수 결과"""

    bias_type: str
    score: float
    severity: str  # "none", "low", "medium", "high"
    details: Dict[str, Any] = field(default_factory=dict)

    def __post_init__(self):
        """심각도 자동 계산"""
        if self.score < 0.05:
            self.severity = "none"
        elif self.score < 0.15:
            self.severity = "low"
        elif self.score < 0.30:
            self.severity = "medium"
        else:
            self.severity = "high"


def evaluate_verbosity_bias(
    test_case: BiasTestCase,
    model: str = "openai/gpt-4.1-mini",
    temperature: float = 0.0,
) -> BiasScore:
    """장황함 편향 평가

    동일한 정보를 담고 있지만 길이가 다른 두 답변의 점수 차이를 측정

    Args:
        test_case: 테스트 케이스 (short_answer, long_answer 필요)
        model: 평가에 사용할 LLM 모델
        temperature: LLM 온도

    Returns:
        BiasScore 객체
    """
    if not test_case.short_answer or not test_case.long_answer:
        raise ValueError("Verbosity bias test requires both short_answer and long_answer")

    print(f"Evaluating Verbosity Bias...")
    print(f"Short answer length: {len(test_case.short_answer)} chars")
    print(f"Long answer length: {len(test_case.long_answer)} chars")

    # 평가 함수 생성
    evaluator = create_llm_evaluator(
        question=test_case.question, context=test_case.context, model=model, temperature=temperature
    )

    # 두 답변 평가
    short_score = evaluator(test_case.short_answer)
    long_score = evaluator(test_case.long_answer)

    # 점수 차이 계산
    diff = abs(short_score - long_score)

    print(f"Short answer score: {short_score:.3f}")
    print(f"Long answer score: {long_score:.3f}")
    print(f"Score difference: {diff:.3f}")

    # 편향 방향 결정
    bias_direction = "none"
    if diff > 0.05:
        bias_direction = "favors_long" if long_score > short_score else "favors_short"

    return BiasScore(
        bias_type="verbosity_bias",
        score=diff,
        severity="",  # __post_init__에서 자동 계산
        details={
            "short_score": short_score,
            "long_score": long_score,
            "bias_direction": bias_direction,
            "short_length": len(test_case.short_answer),
            "long_length": len(test_case.long_answer),
        },
    )


def evaluate_position_bias(
    test_case: BiasTestCase,
    model: str = "openai/gpt-4.1-mini",
    temperature: float = 0.0,
    num_trials: int = 3,
) -> BiasScore:
    """위치 편향 평가

    동일한 답변을 다른 순서로 평가하여 위치 편향 측정

    Args:
        test_case: 테스트 케이스 (answer_A, answer_B 필요)
        model: 평가에 사용할 LLM 모델
        temperature: LLM 온도
        num_trials: 반복 시행 횟수

    Returns:
        BiasScore 객체
    """
    if not test_case.answer_A or not test_case.answer_B:
        raise ValueError("Position bias test requires both answer_A and answer_B")

    print(f"Evaluating Position Bias ({num_trials} trials)...")

    # 여러 번 시행하여 평균 계산 (랜덤성 완화)
    diffs = []

    for trial in range(num_trials):
        print(f"Trial {trial + 1}/{num_trials}... ", end="")

        # 블라인드 평가 수행
        results = blind_evaluation(
            answers=[test_case.answer_A, test_case.answer_B],
            question=test_case.question,
            context=test_case.context,
            model=model,
            temperature=temperature,
            verbose=False,
        )

        score_A = results[0]["score"]
        score_B = results[1]["score"]
        diff = abs(score_A - score_B)
        diffs.append(diff)

        print(f"Diff: {diff:.3f}")

    # 평균 점수 차이
    avg_diff = sum(diffs) / len(diffs)

    print(f"Average score difference: {avg_diff:.3f}")

    return BiasScore(
        bias_type="position_bias",
        score=avg_diff,
        severity="",  # __post_init__에서 자동 계산
        details={
            "num_trials": num_trials,
            "trial_diffs": diffs,
            "std_dev": (sum((d - avg_diff) ** 2 for d in diffs) / len(diffs)) ** 0.5,
        },
    )


def calculate_bias_score(
    test_cases: List[BiasTestCase],
    model: str = "openai/gpt-4.1-mini",
    temperature: float = 0.0,
) -> Dict[str, BiasScore]:
    """편향 점수 계산 (프로덕션 버전)

    Args:
        test_cases: 테스트 케이스 리스트
        model: 평가에 사용할 LLM 모델
        temperature: LLM 온도

    Returns:
        편향 유형별 BiasScore 딕셔너리
    """
    bias_scores = {}

    for test_case in test_cases:
        print("\n" + "=" * 60)
        print(f"Test Type: {test_case.test_type}")
        print("=" * 60)

        try:
            if test_case.test_type == "verbosity":
                bias_score = evaluate_verbosity_bias(test_case, model, temperature)
                bias_scores["verbosity_bias"] = bias_score

            elif test_case.test_type == "position":
                bias_score = evaluate_position_bias(test_case, model, temperature)
                bias_scores["position_bias"] = bias_score

            else:
                print(f"Warning: Unknown test type '{test_case.test_type}', skipping")
                continue

        except Exception as e:
            print(f"Error evaluating {test_case.test_type}: {e}")
            continue

    return bias_scores


# 프로덕션 테스트 실행
print("## 9. Bias Score Calculation (Production)")
print("=" * 60)

# 테스트 케이스 정의
production_test_cases = [
    BiasTestCase(
        question=verbosity_test["question"],
        context=verbosity_test["context"],
        short_answer=verbosity_test["short_answer"],
        long_answer=verbosity_test["long_answer"],
        test_type="verbosity",
    ),
    BiasTestCase(
        question=position_test["question"],
        context="VPN 연결 문제 해결을 위해서는 네트워크 설정 확인, VPN 클라이언트 재시작, IT 지원팀 문의가 필요합니다.",
        answer_A=position_test["answer_A"],
        answer_B=position_test["answer_B"],
        test_type="position",
    ),
]

# 편향 점수 계산
bias_scores = calculate_bias_score(
    test_cases=production_test_cases, model="openai/gpt-4.1-mini", temperature=0.0
)

# 결과 요약
print("\n" + "=" * 60)
print("Bias Score Summary")
print("=" * 60)

for bias_type, bias_score in bias_scores.items():
    print(f"\n{bias_type.upper()}:")
    print(f"  Score: {bias_score.score:.3f}")
    print(f"  Severity: {bias_score.severity}")

    if bias_score.severity == "none":
        print("  Assessment: No significant bias detected")
    elif bias_score.severity == "low":
        print("  Assessment: Minor bias present, acceptable for most use cases")
    elif bias_score.severity == "medium":
        print("  Assessment: Moderate bias detected, consider mitigation strategies")
    else:
        print("  Assessment: Significant bias detected, mitigation required")

    print(f"  Details: {bias_score.details}")

print("\n" + "=" * 60)
print("Production Notes:")
print("- All evaluations use actual LLM calls")
print("- Blind evaluation employed for position bias")
print("- Multiple trials used for statistical robustness")
print("- Severity levels guide mitigation decisions")

📊 편향 점수 (낮을수록 좋음)

verbosity_bias: 0.033
  ✅ 편향 거의 없음

